In [165]:
import pandas as pd
from geopy.geocoders import Photon
from geopy.geocoders import Nominatim
from unidecode import unidecode
import unidecode


In [2]:
import geopandas as gpd

In [3]:
import numpy as np
from shapely.geometry import Point, Polygon

In [24]:
# Cargar el archivo CSV
#datos_csv = pd.read_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones.csv')


In [ ]:
# Cambiar las etiquetas de 'Mexico' a 'Estado de Mexico' en la columna 'estado'
#datos_csv['estado'] = datos_csv['estado'].replace('Mexico', 'Estado de Mexico')

In [ ]:

# Corrección en la importación de unidecode
#def quitar_acentos(valor):
#    if isinstance(valor, str):  # Solo aplica a cadenas de texto
#        return unidecode(valor)  # Uso correcto de unidecode
#    return valor

# Aplicar la función quitar_acentos a todas las columnas del DataFrame
#for columna in datos_csv.columns:
#   datos_csv[columna] = datos_csv[columna].apply(quitar_acentos)



In [ ]:
# Definir las listas de estados para cada ruta, asegurándose de que todas las entradas están correctamente formateadas como strings
estados_ruta_golfo = ['Tabasco', 'Nuevo Leon', 'Veracruz', 'Tamaulipas']
estados_ruta_pacifico = ['Guerrero', 'Michoacan', 'Jalisco', 'Nayarit', 'Sinaloa', 'Sonora', 'Baja California', 'Baja California Sur','Colima']
estados_ruta_centro = ['San Luis Potosi','Estado de Mexico', 'Tlaxcala', 'Ciudad de Mexico', 'Puebla', 'Morelos', 'Hidalgo', 'Zacatecas', 'Durango', 'Chihuahua', 'Coahuila', 'Queretaro', 'Aguascalientes', 'Guanajuato']
ruta_sur = ['Chiapas', 'Oaxaca', 'Tabasco', 'Campeche', 'Quintana Roo', 'Yucatan']

# Actualizar la función para asignar la ruta basada en el estado
#def asignar_ruta(estado):
#    estado = estado.strip()  # Eliminar espacios en blanco al principio y al final
#    if estado in estados_ruta_golfo:
#        return 'Ruta del Golfo'
#    elif estado in estados_ruta_pacifico:
#        return 'Ruta del Pacifico'
#    elif estado in estados_ruta_centro:
#        return 'Ruta Centro'
#    elif estado in ruta_sur:
#        return 'Ruta Sur'
#    else:
#        return 'Ruta no identificada'  # Para estados que no coinciden con las listas

# Aplicar la función al DataFrame para crear una nueva columna 'Ruta'
#datos_csv['ruta'] = datos_csv['estado'].apply(asignar_ruta)

In [ ]:
# Reordenar las columnas para colocar 'Estado' después de 'Ciudad'
#column_order = ['ciudad', 'estado', 'ruta','detenciones']

# Aplicar el nuevo orden de columnas al DataFrame
#datos_csv = datos_csv[column_order]

In [ ]:
# Obtenemos los valores únicos de la columna 'ruta'
#valores_unicos_ruta = datos_csv['ruta'].unique()

# Imprimimos los valores únicos
#print(valores_unicos_ruta)

In [ ]:
# Initialize the geolocator using OSM Nominatim
#geolocator = Photon(user_agent="measurement", timeout=10)

# Define a function to get latitude and longitude
#def get_lat_lon(address):
#    try:
#        location = geolocator.geocode(address)
#        if location:
#            return location.latitude, location.longitude
#        else:
#            return '', ''  # Return empty strings if location is not found
#    except AttributeError:
#        return '', ''

# Apply the function to the 'City' column and split the results into 'Latitude' and 'Longitude' columns
#datos_csv[['latitud', 'longitud']] = datos_csv['ciudad'].apply(lambda x: pd.Series(get_lat_lon(x)))

In [ ]:
# Especifica la ruta donde deseas guardar el archivo CSV
#ruta_archivo_csv = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv'

# Guardar el DataFrame en el archivo CSV especificado
#datos_csv.to_csv(ruta_archivo_csv, index=False)

#print(f"El DataFrame ha sido guardado exitosamente en: {ruta_archivo_csv}")


In [132]:
# Cargar el archivo CSV
#datos_csv1 = pd.read_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv')


In [ ]:
# Convertir la columna 'detenciones' a entero
#datos_csv1['detenciones'] = pd.to_numeric(datos_csv1['detenciones'], errors='coerce').fillna(0).astype(int)

In [ ]:
# Identificar los índices de la primera fila de cada categoría en 'estado'
#indices_a_eliminar = datos_csv1.groupby('estado').head(1).index

# Eliminar esas filas del DataFrame
#datos_csv1 = datos_csv1.drop(indices_a_eliminar)




In [ ]:
# Verificar los cambios
#print(datos_csv1.head())


In [ ]:
# Agregar la columna 'tipo' con la etiqueta 'Presencia Migración Irregular' para todas las filas
#datos_csv1['tipo'] = 'Presencia Migración Irregular'

# Verificar los cambios mostrando las primeras filas del DataFrame
#print(datos_csv1.head())


In [147]:
# Especifica la ruta donde deseas guardar el archivo CSV
#ruta_archivo_csv1 = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv'

# Guardar el DataFrame en el archivo CSV especificado
#datos_csv1.to_csv(ruta_archivo_csv1, index=False)

#print(f"El DataFrame ha sido guardado exitosamente en: {ruta_archivo_csv1}")


### Grid Shape

Para crear un grid con polígonos geoespaciales y asignar a cada punto según su latitud y longitud a un polígono correspondiente, se puede utilizar la biblioteca geopandas en Python, que extiende las capacidades de pandas para trabajar con datos espaciales. También usaremos shapely para manipular y analizar datos geométricos planos, y numpy para manejar operaciones numéricas.



Crear el Grid de Polígonos: Primero, generamos un conjunto de polígonos que representen cada celda del grid sobre el área de interés (en este caso, México). Cada polígono puede ser identificado de manera única, por ejemplo, usando una convención de nomenclatura como PG1, PG2, etc.

In [96]:
def crear_grid(xmin, xmax, ymin, ymax, ancho_celda, alto_celda):
    cols = list(np.arange(xmin, xmax, ancho_celda))
    rows = list(np.arange(ymin, ymax, alto_celda))
    poligonos = []
    ids = []

    id_contador = 1  # Iniciar contador para los ID de polígonos
    for x in cols:
        for y in rows:
            poligonos.append(Polygon([(x, y), (x+ancho_celda, y), (x+ancho_celda, y-alto_celda), (x, y-alto_celda)]))
            ids.append(f"PG{id_contador}")  # Asignar un ID único a cada polígono
            id_contador += 1

    grid = gpd.GeoDataFrame({'ID_Poligono': ids, 'geometry': poligonos})
    return grid

# Generar el grid con los límites aproximados de México y celdas de 1 grado por 1 grado
#xmin, ymin, xmax, ymax = -120, 10, -82, 37
#grid = crear_grid(xmin, xmax, ymin, ymax, 1, 1)

# Ahora, 'grid' es un GeoDataFrame que contiene el 'ID_Poligono' y su 'geometry'
#print(grid.head())

  ID_Poligono                                           geometry
0         PG1  POLYGON ((-120.00000 10.00000, -119.00000 10.0...
1         PG2  POLYGON ((-120.00000 11.00000, -119.00000 11.0...
2         PG3  POLYGON ((-120.00000 12.00000, -119.00000 12.0...
3         PG4  POLYGON ((-120.00000 13.00000, -119.00000 13.0...
4         PG5  POLYGON ((-120.00000 14.00000, -119.00000 14.0...


In [134]:
# Cargar el archivo CSV
#datos_csv1 = pd.read_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv')


In [135]:
# Actualizar Campeche, Campeche
datos_csv1.loc[(datos_csv1['ciudad'] == 'Campeche, Campeche'), 'latitud'] = 19.830285751988043
datos_csv1.loc[(datos_csv1['ciudad'] == 'Campeche, Campeche'), 'longitud'] = -90.53551463998544

# Actualizar Colima, Colima
datos_csv1.loc[(datos_csv1['ciudad'] == 'Colima, Colima'), 'latitud'] = 19.24487815537049
datos_csv1.loc[(datos_csv1['ciudad'] == 'Colima, Colima'), 'longitud'] = -103.7250306766154

# Actualizar Puebla, Puebla
datos_csv1.loc[(datos_csv1['ciudad'] == 'Puebla, Puebla'), 'latitud'] = 19.038406227976886
datos_csv1.loc[(datos_csv1['ciudad'] == 'Puebla, Puebla'), 'longitud'] = -98.2069491554557


datos_csv1.loc[(datos_csv1['ciudad'] == 'Zacatecas, Zacatecas'), 'latitud'] = 19.039018782328817
datos_csv1.loc[(datos_csv1['ciudad'] == 'Zacatecas, Zacatecas'), 'longitud'] = -98.20248616428334

In [136]:
datos_csv1.head(30)

,ciudad,estado,ruta,detenciones,latitud,longitud,tipo,ID_Poligono,geometry,codigo_ruta,contador_xy,codigo_unico
0,"Ensenada, Baja California",Baja California,Ruta del Pacifico,29,31.865889,-116.602983,Presencia Migración Irregular,PG104,"POLYGON ((-117 32, -116 32, -116 31, -117 31, ...",P,1,P-PG104-1
1,"Playas de Rosarito, Baja California",Baja California,Ruta del Pacifico,38,32.258129,-116.873307,Presencia Migración Irregular,PG105,"POLYGON ((-117 33, -116 33, -116 32, -117 32, ...",P,1,P-PG105-1
2,"Tecate, Baja California",Baja California,Ruta del Pacifico,594,32.565383,-116.629879,Presencia Migración Irregular,PG105,"POLYGON ((-117 33, -116 33, -116 32, -117 32, ...",P,2,P-PG105-2
3,"Mexicali, Baja California",Baja California,Ruta del Pacifico,0,32.640525,-115.474899,Presencia Migración Irregular,PG132,"POLYGON ((-116 33, -115 33, -115 32, -116 32, ...",P,1,P-PG132-1
4,"San Felipe, Baja California",Baja California,Ruta del Pacifico,4,31.026953,-114.839607,Presencia Migración Irregular,PG158,"POLYGON ((-115 32, -114 32, -114 31, -115 31, ...",P,1,P-PG158-1
5,"San Luis Rio Colorado, Sonora",Sonora,Ruta del Pacifico,67,32.451796,-114.765254,Presencia Migración Irregular,PG159,"POLYGON ((-115 33, -114 33, -114 32, -115 32, ...",P,1,P-PG159-1
6,"Puerto Penasco, Sonora",Sonora,Ruta del Pacifico,1,31.317087,-113.531819,Presencia Migración Irregular,PG185,"POLYGON ((-114 32, -113 32, -113 31, -114 31, ...",P,1,P-PG185-1
7,"Caborca, Sonora",Sonora,Ruta del Pacifico,414,30.716601,-112.156260,Presencia Migración Irregular,PG211,"POLYGON ((-113 31, -112 31, -112 30, -113 30, ...",P,1,P-PG211-1
8,"General Plutarco Elias Calles, Sonora",Sonora,Ruta del Pacifico,0,31.694984,-112.711749,Presencia Migración Irregular,PG212,"POLYGON ((-113 32, -112 32, -112 31, -113 31, ...",P,1,P-PG212-1
9,"Santa Ana, Sonora",Sonora,Ruta del Pacifico,0,30.541092,-111.118488,Presencia Migración Irregular,PG238,"POLYGON ((-112 31, -111 31, -111 30, -112 30, ...",P,1,P-PG238-1


In [150]:
# Eliminar las columnas del DataFrame
#datos_csv1 = datos_csv1.drop(columns=['ID_Poligono','geometry', 'codigo_ruta', 'contador_xy', 'codigo_unico'])

# Verificación: Muestra las primeras filas para asegurarse de que las columnas han sido eliminadas
#print(datos_csv1.head())


In [139]:
# Convertir la columna 'detenciones' a entero
#datos_csv1['detenciones'] = pd.to_numeric(datos_csv1['detenciones'], errors='coerce').fillna(0).astype(int)

In [140]:
# Paso 2: Asignar Puntos a Polígonos
puntos = [Point(xy) for xy in zip(datos_csv1.longitud, datos_csv1.latitud)]
geo_df = gpd.GeoDataFrame(datos_csv1, geometry=puntos)

# Realizar la asignación
geo_df = gpd.sjoin(geo_df, grid, how='left', op='within')

# Paso 3: Actualizar DataFrame
datos_csv1['ID_Poligono'] = geo_df['ID_Poligono']

/Users/pablouriarte/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3377: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [141]:
# Asegurándose de que datos_csv2 es un GeoDataFrame
if not isinstance(datos_csv1, gpd.GeoDataFrame):
    # Si datos_csv2 ya tiene una columna 'geometry', úsala; de lo contrario, se necesita definir una
    if 'geometry' in datos_csv1.columns:
        datos_csv1 = gpd.GeoDataFrame(datos_csv1, geometry='geometry')
    else:
        # Aquí se asume que no hay una columna 'geometry' existente y que quizás no necesites definir una en este momento
        datos_csv1 = gpd.GeoDataFrame(datos_csv1)

# Realizar el merge (unión) entre datos_csv2 y grid usando 'ID_Poligono' como clave
datos_csv1 = datos_csv1.merge(grid[['ID_Poligono', 'geometry']], on='ID_Poligono', how='left')


In [148]:
datos_csv1.tail(20)

,ciudad,estado,ruta,detenciones,latitud,longitud,tipo,ID_Poligono,geometry,codigo_ruta,contador_xy,codigo_unico
369,"Balancan, Tabasco",Tabasco,Ruta del Golfo,0,17.797517,-91.345859,Presencia Migración Irregular,PG765,"POLYGON ((-92.00000 18.00000, -91.00000 18.000...",G,4,G-PG765-4
370,"Carmen, Campeche",Campeche,Ruta Sur,255,18.651739,-91.814920,Presencia Migración Irregular,PG766,"POLYGON ((-92.00000 19.00000, -91.00000 19.000...",S,1,S-PG766-1
371,"Tijuana, Baja California",Baja California,Ruta del Pacifico,0,32.531740,-117.019529,Presencia Migración Irregular,PG78,"POLYGON ((-118.00000 33.00000, -117.00000 33.0...",P,1,P-PG78-1
372,"Escarcega, Campeche",Campeche,Ruta Sur,0,18.606934,-90.738582,Presencia Migración Irregular,PG793,"POLYGON ((-91.00000 19.00000, -90.00000 19.000...",S,1,S-PG793-1
373,"Champoton, Campeche",Campeche,Ruta Sur,150,19.343965,-90.724274,Presencia Migración Irregular,PG794,"POLYGON ((-91.00000 20.00000, -90.00000 20.000...",S,1,S-PG794-1
374,"Seybaplaya, Campeche",Campeche,Ruta Sur,8,19.603372,-90.625367,Presencia Migración Irregular,PG794,"POLYGON ((-91.00000 20.00000, -90.00000 20.000...",S,2,S-PG794-2
375,"Campeche, Campeche",Campeche,Ruta Sur,0,19.830286,-90.535515,Presencia Migración Irregular,PG794,"POLYGON ((-91.00000 20.00000, -90.00000 20.000...",S,3,S-PG794-3
376,"Tenabo, Campeche",Campeche,Ruta Sur,22,20.040661,-90.225496,Presencia Migración Irregular,PG795,"POLYGON ((-91.00000 21.00000, -90.00000 21.000...",S,1,S-PG795-1
377,"Calkini, Campeche",Campeche,Ruta Sur,1,20.369512,-90.051078,Presencia Migración Irregular,PG795,"POLYGON ((-91.00000 21.00000, -90.00000 21.000...",S,2,S-PG795-2
378,"Calakmul, Campeche",Campeche,Ruta Sur,194,18.107341,-89.808328,Presencia Migración Irregular,PG820,"POLYGON ((-90.00000 19.00000, -89.00000 19.000...",S,1,S-PG820-1


Asignar Puntos a Polígonos: Luego, para cada punto dado por sus coordenadas de latitud y longitud en datos_csv1, determinamos en qué polígono del grid se encuentra y asignamos el identificador de ese polígono al punto.


el codigo de un punto se miraria algo asi: S-PG#-1

Para asignar un código que combine el código de ruta por estado, el identificador del polígono al que pertenece cada punto y un número ascendente basado en el orden de las coordenadas XY dentro de cada polígono

por estado, si:
    'Ruta Sur': 'S',
    'Ruta del Pacifico': 'P',
    'Ruta Centro': 'C',
    'Ruta del Golfo': 'G'
    
por ID_Poligono: el poligono a que pertenece

y por orden asendientes de su coordenada xy dentro de cada poligono

In [143]:
# Primero, mapeamos la columna 'ruta' a su código correspondiente
ruta_to_code = {
    'Ruta Sur': 'S',
    'Ruta del Pacifico': 'P',
    'Ruta Centro': 'C',
    'Ruta del Golfo': 'G'
}
datos_csv1['codigo_ruta'] = datos_csv1['ruta'].map(ruta_to_code)

# Asumiendo que 'datos_csv2' ya tiene una columna 'ID_Poligono',
# vamos a crear una columna que identifique el orden XY dentro de cada polígono
# Para esto, primero ordenamos el DataFrame
datos_csv1.sort_values(by=['ID_Poligono', 'latitud', 'longitud'], ascending=[True, True, True], inplace=True)

# Creamos una nueva columna que será el contador dentro de cada grupo de 'ID_Poligono'
datos_csv1['contador_xy'] = datos_csv1.groupby('ID_Poligono').cumcount() + 1

# Ahora, combinamos todo para crear el código único
datos_csv1['codigo_unico'] = datos_csv1['codigo_ruta'] + '-' + datos_csv1['ID_Poligono'] + '-' + datos_csv1['contador_xy'].astype(str)

# Revisamos el resultado
print(datos_csv1[['ciudad', 'estado', 'ruta', 'ID_Poligono', 'codigo_unico']])


                                  ciudad           estado               ruta  \
0              Ensenada, Baja California  Baja California  Ruta del Pacifico   
1    Playas de Rosarito, Baja California  Baja California  Ruta del Pacifico   
2                Tecate, Baja California  Baja California  Ruta del Pacifico   
3              Mexicali, Baja California  Baja California  Ruta del Pacifico   
4            San Felipe, Baja California  Baja California  Ruta del Pacifico   
..                                   ...              ...                ...   
384                  Tulum, Quintana Roo     Quintana Roo           Ruta Sur   
385            Solidaridad, Quintana Roo     Quintana Roo           Ruta Sur   
386                Cozumel, Quintana Roo     Quintana Roo           Ruta Sur   
387          Benito Juarez, Quintana Roo     Quintana Roo           Ruta Sur   
388           Isla Mujeres, Quintana Roo     Quintana Roo           Ruta Sur   

    ID_Poligono codigo_unico  
0       

Actualizar DataFrame: Finalmente, agregamos una nueva columna al DataFrame datos_csv1 para almacenar el identificador del polígono del grid al que pertenece cada punto.

In [144]:
# Obtener el conteo de los valores únicos en la columna 'ID_Poligono'
conteo_id_poligono = datos_csv1['ID_Poligono'].value_counts()

# Mostrar el conteo
print(conteo_id_poligono)



ID_Poligono
PG578    38
PG551    19
PG630    15
PG530    15
PG631    12
         ..
PG450     1
PG446     1
PG426     1
PG421     1
PG554     1
Name: count, Length: 121, dtype: int64


In [146]:
# Especifica la ruta donde deseas guardar el archivo CSV
ruta_archivo_csv4 = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/grid.csv'

# Guardar el DataFrame en el archivo CSV especificado
grid.to_csv(ruta_archivo_csv4, index=False)

print(f"El DataFrame ha sido guardado exitosamente en: {ruta_archivo_csv3}")

El DataFrame ha sido guardado exitosamente en: /Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv


In [149]:
grid

,ID_Poligono,geometry
0,PG1,"POLYGON ((-120.00000 10.00000, -119.00000 10.0..."
1,PG2,"POLYGON ((-120.00000 11.00000, -119.00000 11.0..."
2,PG3,"POLYGON ((-120.00000 12.00000, -119.00000 12.0..."
3,PG4,"POLYGON ((-120.00000 13.00000, -119.00000 13.0..."
4,PG5,"POLYGON ((-120.00000 14.00000, -119.00000 14.0..."
...,...,...
1021,PG1022,"POLYGON ((-83.00000 32.00000, -82.00000 32.000..."
1022,PG1023,"POLYGON ((-83.00000 33.00000, -82.00000 33.000..."
1023,PG1024,"POLYGON ((-83.00000 34.00000, -82.00000 34.000..."
1024,PG1025,"POLYGON ((-83.00000 35.00000, -82.00000 35.000..."


In [153]:
datos_csv2

,ciudad,estado,ruta,detenciones,latitud,longitud,tipo,ID_Poligono,geometry,codigo_ruta,contador_xy,codigo_unico
0,"Ensenada, Baja California",Baja California,Ruta del Pacifico,29,31.865889,-116.602983,Presencia Migración Irregular,PG104,"POLYGON ((-117 32, -116 32, -116 31, -117 31, ...",P,1,P-PG104-1
1,"Playas de Rosarito, Baja California",Baja California,Ruta del Pacifico,38,32.258129,-116.873307,Presencia Migración Irregular,PG105,"POLYGON ((-117 33, -116 33, -116 32, -117 32, ...",P,1,P-PG105-1
2,"Tecate, Baja California",Baja California,Ruta del Pacifico,594,32.565383,-116.629879,Presencia Migración Irregular,PG105,"POLYGON ((-117 33, -116 33, -116 32, -117 32, ...",P,2,P-PG105-2
3,"Mexicali, Baja California",Baja California,Ruta del Pacifico,0,32.640525,-115.474899,Presencia Migración Irregular,PG132,"POLYGON ((-116 33, -115 33, -115 32, -116 32, ...",P,1,P-PG132-1
4,"San Felipe, Baja California",Baja California,Ruta del Pacifico,4,31.026953,-114.839607,Presencia Migración Irregular,PG158,"POLYGON ((-115 32, -114 32, -114 31, -115 31, ...",P,1,P-PG158-1
...,...,...,...,...,...,...,...,...,...,...,...,...
384,"Tulum, Quintana Roo",Quintana Roo,Ruta Sur,13,20.169297,-87.449548,Presencia Migración Irregular,PG876,"POLYGON ((-88 21, -87 21, -87 20, -88 20, -88 ...",S,1,S-PG876-1
385,"Solidaridad, Quintana Roo",Quintana Roo,Ruta Sur,178,20.584375,-87.503453,Presencia Migración Irregular,PG876,"POLYGON ((-88 21, -87 21, -87 20, -88 20, -88 ...",S,2,S-PG876-2
386,"Cozumel, Quintana Roo",Quintana Roo,Ruta Sur,24,20.432060,-86.920691,Presencia Migración Irregular,PG903,"POLYGON ((-87 21, -86 21, -86 20, -87 20, -87 ...",S,1,S-PG903-1
387,"Benito Juarez, Quintana Roo",Quintana Roo,Ruta Sur,0,21.076681,-86.926335,Presencia Migración Irregular,PG904,"POLYGON ((-87 22, -86 22, -86 21, -87 21, -87 ...",S,1,S-PG904-1


### Arreglamos detenciones

reordenar mi df datos_csv2, usando su columna estado, sort en orden alfabetico y resetear el index

In [152]:
# Cargar el archivo CSV
datos_csv2 = pd.read_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv')


In [155]:
# Ordenamos el DataFrame por la columna 'estado' en orden alfabético
datos_csv2_ordenado = datos_csv2.sort_values(by='estado')

# Reseteamos el índice del DataFrame ordenado
datos_csv2_ordenado.reset_index(drop=True, inplace=True)



In [157]:
# Especifica la ruta donde deseas guardar el archivo CSV
ruta_archivo_csv5 = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv'

# Guardar el DataFrame en el archivo CSV especificado
datos_csv2_ordenado.to_csv(ruta_archivo_csv5, index=False)

print(f"El DataFrame ha sido guardado exitosamente en: {ruta_archivo_csv5}")

El DataFrame ha sido guardado exitosamente en: /Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv


In [158]:
# Cargar el archivo CSV
datos_csv3 = pd.read_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv')


#### AGREGamos columna con los datos de detenciones, procesamos y unimos

In [160]:
# Cargar el archivo CSV
datos_detenciones_2023 = pd.read_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones.csv')


In [ ]:
datos_detenciones_2023 = datos_detenciones_2023.dropna()

In [161]:
datos_detenciones_2023.head()

,Unnamed: 0,Unnamed: 1
0,Aguascalientes,559
1,"Aguascalientes, Ags.",423
2,"Cosío, Ags.",98
3,"Jesús María, Ags.",20
4,"San Francisco de los Romo, Ags.",18


In [168]:
# Cambiamos los nombres de las columnas usando el método 'rename'
datos_detenciones_2023 = datos_detenciones_2023.rename(columns={'Unnamed: 0': 'ciudad', 'Unnamed: 1': 'detenciones'})



In [170]:
# Función para eliminar acentos en una cadena de texto, manejando valores no-cadena
def quitar_acentos(texto):
    # Si el valor es una cadena, aplicamos unidecode
    if isinstance(texto, str):
        return unidecode.unidecode(texto)
    # Si no es una cadena, lo dejamos tal cual (esto incluye NaN y números)
    return texto

# Aplicamos la función a todas las columnas de tipo object en el DataFrame
for columna in datos_detenciones_2023.select_dtypes(include=['object']).columns:
    datos_detenciones_2023[columna] = datos_detenciones_2023[columna].apply(quitar_acentos)



In [176]:
# Función para detectar si un nombre de ciudad tiene caracteres especiales (no es alfanumérico)
def tiene_caracteres_especiales(cadena):
    return not cadena.replace(" ", "").isalnum()

# Crea una nueva columna 'estado' inicialmente llena con NaN
datos_detenciones_2023['estado'] = pd.NA

# Inicializamos una variable para almacenar el estado actual mientras iteramos
estado_actual = None

# Iteramos sobre las filas del DataFrame
for index, row in datos_detenciones_2023.iterrows():
    # Si la ciudad no tiene caracteres especiales, actualizamos estado_actual
    if not tiene_caracteres_especiales(row['ciudad']):
        estado_actual = row['ciudad']
    # Asignamos el estado actual a la columna 'estado'
    datos_detenciones_2023.at[index, 'estado'] = estado_actual

In [178]:
# Filtramos el DataFrame para mantener solo las filas donde la columna 'ciudad' tiene caracteres especiales
datos_detenciones_2023 = datos_detenciones_2023[datos_detenciones_2023['ciudad'].apply(tiene_caracteres_especiales)]


In [180]:
# Asegúrate de que 'datos_detenciones_2023' es el DataFrame actual después de eliminar las filas con NaN.
datos_detenciones_2023 = datos_detenciones_2023[['ciudad', 'estado', 'detenciones']]


In [182]:
# Reemplazamos el contenido de la columna 'ciudad' después de la primera coma por el valor de la columna 'estado'
datos_detenciones_2023['ciudad'] = datos_detenciones_2023.apply(lambda fila: fila['ciudad'].split(',', 1)[0] + ", " + fila['estado'], axis=1)


In [183]:
# Verificamos el cambio
print(datos_detenciones_2023.head(20))

                                       ciudad               estado detenciones
1              Aguascalientes, Aguascalientes       Aguascalientes         423
2                       Cosio, Aguascalientes       Aguascalientes          98
3                 Jesus Maria, Aguascalientes       Aguascalientes          20
4   San Francisco de los Romo, Aguascalientes       Aguascalientes          18
7                   Ensenada, Baja California      Baja California          29
8                   Mexicali, Baja California      Baja California       5 706
9         Playas de Rosarito, Baja California      Baja California          38
10                San Felipe, Baja California      Baja California           4
11                    Tecate, Baja California      Baja California         594
12                   Tijuana, Baja California      Baja California      27 568
15                La Paz, Baja California Sur  Baja California Sur         342
16             Los Cabos, Baja California Sur  Baja 

In [185]:
# Especifica la ruta donde deseas guardar el archivo CSV
ruta_archivo_csv6 = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_2023.csv'

# Guardar el DataFrame en el archivo CSV especificado
datos_detenciones_2023.to_csv(ruta_archivo_csv6, index=False)

print(f"El DataFrame ha sido guardado exitosamente en: {ruta_archivo_csv6}")

El DataFrame ha sido guardado exitosamente en: /Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_2023.csv


In [187]:
# Realizamos un merge entre datos_csv3 y datos_detenciones_2023 basándonos en 'ciudad'
datos_combinados = pd.merge(datos_csv3, datos_detenciones_2023[['ciudad', 'detenciones']],
                            on='ciudad', how='left', suffixes=('', '_nueva'))


In [188]:
# Actualizamos la columna 'detenciones' en datos_csv3 con los valores de 'detenciones_nueva'
datos_csv3['detenciones'] = datos_combinados['detenciones_nueva']


In [191]:
# Verificamos el cambio
print(datos_csv3.head())

                                      ciudad           estado  \
0             Aguascalientes, Aguascalientes   Aguascalientes   
1                      Cosio, Aguascalientes   Aguascalientes   
2  San Francisco de los Romo, Aguascalientes   Aguascalientes   
3                Jesus Maria, Aguascalientes   Aguascalientes   
4                   Tijuana, Baja California  Baja California   

                ruta detenciones    latitud    longitud  \
0        Ruta Centro         423  22.000000 -102.500000   
1        Ruta Centro          98  22.370515 -102.312009   
2        Ruta Centro          18  22.074628 -102.270713   
3        Ruta Centro          20  21.934555 -102.468452   
4  Ruta del Pacifico      27 568  32.531740 -117.019529   

                            tipo ID_Poligono  \
0  Presencia Migración Irregular       PG473   
1  Presencia Migración Irregular       PG473   
2  Presencia Migración Irregular       PG473   
3  Presencia Migración Irregular       PG472   
4  Presencia M

In [192]:
# Especifica la ruta donde deseas guardar el archivo CSV
ruta_archivo_csv7 = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv'

# Guardar el DataFrame en el archivo CSV especificado
datos_csv3.to_csv(ruta_archivo_csv7, index=False)

print(f"El DataFrame ha sido guardado exitosamente en: {ruta_archivo_csv7}")

El DataFrame ha sido guardado exitosamente en: /Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv


### Hacemos segundo df con geometry_range

In [193]:
datos_csv3.head()

,ciudad,estado,ruta,detenciones,latitud,longitud,tipo,ID_Poligono,geometry,codigo_ruta,contador_xy,codigo_unico
0,"Aguascalientes, Aguascalientes",Aguascalientes,Ruta Centro,423,22.000000,-102.500000,Presencia Migración Irregular,PG473,"POLYGON ((-103 23, -102 23, -102 22, -103 22, ...",C,1,C-PG473-1
1,"Cosio, Aguascalientes",Aguascalientes,Ruta Centro,98,22.370515,-102.312009,Presencia Migración Irregular,PG473,"POLYGON ((-103 23, -102 23, -102 22, -103 22, ...",C,3,C-PG473-3
2,"San Francisco de los Romo, Aguascalientes",Aguascalientes,Ruta Centro,18,22.074628,-102.270713,Presencia Migración Irregular,PG473,"POLYGON ((-103 23, -102 23, -102 22, -103 22, ...",C,2,C-PG473-2
3,"Jesus Maria, Aguascalientes",Aguascalientes,Ruta Centro,20,21.934555,-102.468452,Presencia Migración Irregular,PG472,"POLYGON ((-103 22, -102 22, -102 21, -103 21, ...",C,1,C-PG472-1
4,"Tijuana, Baja California",Baja California,Ruta del Pacifico,27 568,32.531740,-117.019529,Presencia Migración Irregular,PG78,"POLYGON ((-118 33, -117 33, -117 32, -118 32, ...",P,1,P-PG78-1


In [196]:
# Seleccionamos las columnas específicas de datos_csv3
datos_csv4 = datos_csv3[['ciudad', 'estado', 'ruta', 'detenciones', 'latitud', 'longitud', 'tipo', 'ID_Poligono', 'codigo_unico']].copy()

# Reseteamos el índice del nuevo DataFrame
datos_csv3.reset_index(drop=True, inplace=True)

# Verificamos el resultado
print(datos_csv4.head())


                                      ciudad           estado  \
0             Aguascalientes, Aguascalientes   Aguascalientes   
1                      Cosio, Aguascalientes   Aguascalientes   
2  San Francisco de los Romo, Aguascalientes   Aguascalientes   
3                Jesus Maria, Aguascalientes   Aguascalientes   
4                   Tijuana, Baja California  Baja California   

                ruta detenciones    latitud    longitud  \
0        Ruta Centro         423  22.000000 -102.500000   
1        Ruta Centro          98  22.370515 -102.312009   
2        Ruta Centro          18  22.074628 -102.270713   
3        Ruta Centro          20  21.934555 -102.468452   
4  Ruta del Pacifico      27 568  32.531740 -117.019529   

                            tipo ID_Poligono codigo_unico  
0  Presencia Migración Irregular       PG473    C-PG473-1  
1  Presencia Migración Irregular       PG473    C-PG473-3  
2  Presencia Migración Irregular       PG473    C-PG473-2  
3  Presencia M

In [197]:
# Asegurándonos de trabajar con el DataFrame correcto, df1
gdf = gpd.GeoDataFrame(datos_csv4, geometry=gpd.points_from_xy(datos_csv4.longitud, datos_csv4.latitud))

# Estableciendo el CRS original a EPSG:4326 para las coordenadas geográficas
gdf.crs = "EPSG:4326"

# Cambiando el CRS a uno que use metros para el cálculo del buffer. Reemplazando el EPSG con un código correcto.
# Nota: El EPSG 6366 parece ser un error. Asegúrate de usar el código EPSG de UTM adecuado para tu zona.
gdf_metros = gdf.to_crs(epsg=6366)  # Reemplaza YOUR_UTM_EPSG_CODE con el código EPSG adecuado

# Aplicando el buffer de 10km para crear una nueva columna 'geometry_range' con los polígonos
gdf_metros['geometry_range'] = gdf_metros.geometry.buffer(10000)

# (Opcional) Si deseas convertir el GeoDataFrame de regreso a lat/long (EPSG:4326) para alguna otra operación,
# y quieres que la columna 'geometry_range' sea la geometría activa,
# debes hacerlo antes de cambiar el CRS de regreso a EPSG:4326.
# gdf_metros.set_geometry('geometry_range', inplace=True)

# Cambiando de regreso a EPSG:4326
gdf_final = gdf_metros.to_crs(epsg=4326)

In [198]:
gdf_final

,ciudad,estado,ruta,detenciones,latitud,longitud,tipo,ID_Poligono,codigo_unico,geometry,geometry_range
0,"Aguascalientes, Aguascalientes",Aguascalientes,Ruta Centro,423,22.000000,-102.500000,Presencia Migración Irregular,PG473,C-PG473-1,POINT (-102.50000 22.00000),"POLYGON ((2018353.402 2505396.238, 2018305.249..."
1,"Cosio, Aguascalientes",Aguascalientes,Ruta Centro,98,22.370515,-102.312009,Presencia Migración Irregular,PG473,C-PG473-3,POINT (-102.31201 22.37052),"POLYGON ((2034061.510 2549327.296, 2034013.357..."
2,"San Francisco de los Romo, Aguascalientes",Aguascalientes,Ruta Centro,18,22.074628,-102.270713,Presencia Migración Irregular,PG473,C-PG473-2,POINT (-102.27071 22.07463),"POLYGON ((2041749.918 2516224.034, 2041701.765..."
3,"Jesus Maria, Aguascalientes",Aguascalientes,Ruta Centro,20,21.934555,-102.468452,Presencia Migración Irregular,PG472,C-PG472-1,POINT (-102.46845 21.93456),"POLYGON ((2022405.596 2498304.969, 2022357.443..."
4,"Tijuana, Baja California",Baja California,Ruta del Pacifico,27 568,32.531740,-117.019529,Presencia Migración Irregular,PG78,P-PG78-1,POINT (-117.01953 32.53174),"POLYGON ((508166.107 3599377.675, 508117.954 3..."
...,...,...,...,...,...,...,...,...,...,...,...
384,"Valladolid, Yucatan",Yucatan,Ruta Sur,5,20.690251,-88.201700,Presencia Migración Irregular,PG849,S-PG849-1,POINT (-88.20170 20.69025),"POLYGON ((3607494.183 2580365.043, 3607446.030..."
385,"Zacatecas, Zacatecas",Zacatecas,Ruta Centro,986,19.039019,-98.202486,Presencia Migración Irregular,PG578,C-PG578-2,POINT (-98.20249 19.03902),"POLYGON ((2516543.276 2215332.307, 2516495.123..."
386,"Calera, Zacatecas",Zacatecas,Ruta Centro,42,22.978480,-102.856223,Presencia Migración Irregular,PG473,C-PG473-5,POINT (-102.85622 22.97848),"POLYGON ((1970070.346 2612482.677, 1970022.193..."
387,"Jerez, Zacatecas",Zacatecas,Ruta Centro,1,22.710593,-103.001473,Presencia Migración Irregular,PG446,C-PG446-1,POINT (-103.00147 22.71059),"POLYGON ((1957797.230 2580711.677, 1957749.077..."


In [199]:
# Especifica la ruta donde deseas guardar el archivo CSV
ruta_archivo_csv7 = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_geometry_range.csv'

# Guardar el DataFrame en el archivo CSV especificado
gdf_final.to_csv(ruta_archivo_csv6, index=False)

print(f"El DataFrame ha sido guardado exitosamente en: {ruta_archivo_csv7}")

El DataFrame ha sido guardado exitosamente en: /Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_geometry_range.csv
